In [ ]:
import os
import yaml

import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf

from model.VGG11 import VGG11
from preprocess.preprocess import load_data

In [ ]:
with open('config/config.yml', 'r') as stream:
    config = yaml.safe_load(stream)

ROOT_DIRECTORY = os.path.dirname(os.path.abspath('__file__'))

# Initialize the model

In [ ]:
input_shape = (config["img_height"], config["img_width"], 3)
model = VGG11(input_shape = input_shape, num_classes = config["num_classes"])

# Load data

In [ ]:
path_to_set = os.path.join(ROOT_DIRECTORY, config["path_to_data"])
path_to_train_csv = os.path.join(ROOT_DIRECTORY, config["path_to_train_csv"])
path_to_test_csv = os.path.join(ROOT_DIRECTORY, config["path_to_test_csv"])
(X_train, y_train) = load_data(path_to_train_csv, path_to_set, config["img_width"], config["img_height"])
(X_test, y_test) = load_data(path_to_test_csv, path_to_set, config["img_width"], config["img_height"])

# Normalize the data

In [ ]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# One-Hot Encode Target value

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, config["num_classes"])
y_test = tf.keras.utils.to_categorical(y_test, config["num_classes"])

# Shuffle train data before splitting

In [ ]:
X_train, y_train = sklearn.utils.shuffle(X_train, y_train, random_state = 0)

# Training

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = config["learning_rate"], decay = config["learning_rate"] / (config["epochs"]))
loss = config["loss_function"]
metrics = config["metrics"]
model.compile(optimizer = optimizer, loss = loss, metrics = [metrics])
fit = model.fit(
  x = X_train,
  y = y_train,
  validation_split = 0.2,
  batch_size = config["batch_size"],
  verbose = 1,
  epochs = config["epochs"])

# Plot loss in each epoch

In [ ]:
loss = fit.history['loss']
val_loss = fit.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Evaluate on test set

In [ ]:
model.evaluate(X_test, y_test, batch_size = config["batch_size"])

# Save model

In [ ]:
model.save_weights(os.path.join(ROOT_DIRECTORY, "../weights/weight.h5"))